In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:08<00:00,  3.70it/s]



eval images shape: (30, 1041, 1302) contain 10205 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 112.0,
 'e_per_adu': 0.4,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/RN_e_venus.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 20838.705 || Time Upd.: 400.7 ms  || BatchNr.: 100
 || Cost: 207.793 || Time Upd.: 373.8 ms  || BatchNr.: 200
 || Cost: 85.398 || Time Upd.: 373.8 ms  || BatchNr.: 300
 || Cost: 52.817 || Time Upd.: 372.3 ms  || BatchNr.: 400
 || Cost: 42.380 || Time Upd.: 373.1 ms  || BatchNr.: 500
 || Cost: 38.617 || Time Upd.: 419.6 ms  || BatchNr.: 600
 || Cost: 31.818 || Time Upd.: 408.4 ms  || BatchNr.: 700
 || Cost: 30.308 || Time Upd.: 409.0 ms  || BatchNr.: 800
 || Cost: 28.206 || Time Upd.: 397.9 ms  || BatchNr.: 900
 || Cost: 25.259 || Time Upd.: 398.2 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8176
JoR: 1.614 || Eff_3d: 56.567 || Jaccard: 0.795 || Factor: 399.441 || RMSE_lat: 49.281 || RMSE_ax: 53.018 || RMSE_x: 37.115

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8272
JoR: 1.816 || Eff_3d: 59.455 || Jaccard: 0.809 || Factor: 344.680 || RMSE_lat: 44.527 || RMSE_ax: 52.892 || RMSE_x: 34.066 || RMSE_y: 28.673 || Cost: 18.872 || Recall: 0.810 || Precision: 0.999 || BatchNr.: 2200 || Time Upd.: 379.2 ms 
 || Count Cost: 14.7003 || Localization Cost: 7.6689 || Bg Cost: 0.0430 || P_locs Cost: 19.2780saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 59.455 || 44.527 || 52.892 || 2200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 7835
JoR: 1.849 || Eff_3d: 60.058 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8099
JoR: 1.875 || Eff_3d: 59.926 || Jaccard: 0.792 || Factor: 317.981 || RMSE_lat: 42.261 || RMSE_ax: 51.466 || RMSE_x: 30.934 || RMSE_y: 28.794 || Cost: 14.896 || Recall: 0.793 || Precision: 0.999 || BatchNr.: 3400 || Time Upd.: 374.9 ms 
 || Count Cost: 14.6011 || Localization Cost: 4.6433 || Bg Cost: 0.0338 || P_locs Cost: 18.5204saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 59.926 || 42.261 || 51.466 || 3400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8169
JoR: 1.875 || Eff_3d: 60.937 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8596
JoR: 1.922 || Eff_3d: 62.805 || Jaccard: 0.838 || Factor: 369.374 || RMSE_lat: 43.627 || RMSE_ax: 45.395 || RMSE_x: 32.580 || RMSE_y: 29.015 || Cost: 13.792 || Recall: 0.840 || Precision: 0.997 || BatchNr.: 4600 || Time Upd.: 376.0 ms 
 || Count Cost: 14.5725 || Localization Cost: 3.9038 || Bg Cost: 0.0293 || P_locs Cost: 18.5075saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 62.805 || 43.627 || 45.395 || 4600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8295
JoR: 1.990 || Eff_3d: 62.658 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8517
JoR: 1.956 || Eff_3d: 62.890 || Jaccard: 0.833 || Factor: 333.739 || RMSE_lat: 42.595 || RMSE_ax: 46.188 || RMSE_x: 30.862 || RMSE_y: 29.358 || Cost: 11.911 || Recall: 0.834 || Precision: 0.999 || BatchNr.: 5800 || Time Upd.: 374.1 ms 
 || Count Cost: 14.1797 || Localization Cost: 2.3722 || Bg Cost: 0.0279 || P_locs Cost: 17.8946saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 62.890 || 42.595 || 46.188 || 5800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8619
JoR: 1.958 || Eff_3d: 63.526 

...

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8851
JoR: 2.072 || Eff_3d: 65.312 || Jaccard: 0.864 || Factor: 340.655 || RMSE_lat: 41.713 || RMSE_ax: 43.223 || RMSE_x: 31.367 || RMSE_y: 27.498 || Cost: 8.414 || Recall: 0.866 || Precision: 0.998 || BatchNr.: 14200 || Time Upd.: 380.5 ms 
 || Count Cost: 13.7593 || Localization Cost: -0.1762 || Bg Cost: 0.0201 || P_locs Cost: 16.9205saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.312 || 41.713 || 43.223 || 14200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 9158
JoR: 1.985 || Eff_3d: 64.43

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8873
JoR: 2.007 || Eff_3d: 63.985 || Jaccard: 0.867 || Factor: 334.532 || RMSE_lat: 43.205 || RMSE_ax: 46.638 || RMSE_x: 32.209 || RMSE_y: 28.797 || Cost: 8.899 || Recall: 0.868 || Precision: 0.999 || BatchNr.: 15400 || Time Upd.: 377.6 ms 
 || Count Cost: 14.2939 || Localization Cost: -0.0418 || Bg Cost: 0.0223 || P_locs Cost: 17.1964saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 63.985 || 43.205 || 46.638 || 15400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8938
JoR: 2.045 || Eff_3d: 65.28

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8730
JoR: 2.085 || Eff_3d: 65.400 || Jaccard: 0.853 || Factor: 332.215 || RMSE_lat: 40.913 || RMSE_ax: 42.247 || RMSE_x: 30.519 || RMSE_y: 27.248 || Cost: 7.919 || Recall: 0.854 || Precision: 0.999 || BatchNr.: 16600 || Time Upd.: 383.7 ms 
 || Count Cost: 13.9822 || Localization Cost: -0.9220 || Bg Cost: 0.0229 || P_locs Cost: 16.8436saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.400 || 40.913 || 42.247 || 16600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8902
JoR: 2.047 || Eff_3d: 65.21

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8764
JoR: 2.064 || Eff_3d: 65.250 || Jaccard: 0.856 || Factor: 340.051 || RMSE_lat: 41.502 || RMSE_ax: 42.354 || RMSE_x: 31.507 || RMSE_y: 27.014 || Cost: 7.603 || Recall: 0.858 || Precision: 0.999 || BatchNr.: 17800 || Time Upd.: 381.0 ms 
 || Count Cost: 14.1573 || Localization Cost: -1.1016 || Bg Cost: 0.0201 || P_locs Cost: 17.0585saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.250 || 41.502 || 42.354 || 17800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8875
JoR: 2.046 || Eff_3d: 65.25

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8905
JoR: 2.019 || Eff_3d: 64.692 || Jaccard: 0.869 || Factor: 341.185 || RMSE_lat: 43.037 || RMSE_ax: 44.080 || RMSE_x: 32.511 || RMSE_y: 28.200 || Cost: 7.512 || Recall: 0.871 || Precision: 0.998 || BatchNr.: 19000 || Time Upd.: 411.2 ms 
 || Count Cost: 13.8276 || Localization Cost: -1.1725 || Bg Cost: 0.0214 || P_locs Cost: 16.8379saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 64.692 || 43.037 || 44.080 || 19000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10205FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10205 ,preds: 8702
JoR: 2.095 || Eff_3d: 65.23

In [11]:
from cellbgnet.utils.plot_funcs import plot_train_record

In [12]:
plot_train_record(model)